# Lab 3

## Question 1

## Question 2

In [2]:
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
import wget


def collect_pdfs():
    base_url = 'http://proceedings.mlr.press/v70/'
    html = urlopen(base_url).read()
    html_page = bs(html) 
    
    collected = []
    
    for link in html_page.find_all('a'):
        if link.get('href').endswith('pdf'):
            try:
                filename = './pdfs/' + link.get('href').split('/')[-1]
                wget.download(link.get('href'), out=filename)
                collected.append(filename)
            except:
                print('Could not download ', link.get('href'))
    
    return collected

In [3]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

def pdf_to_text(filename):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    with open(filename, 'rb') as pdf:
        for page in PDFPage.get_pages(pdf):
            interpreter.process_page(page)

    text = retstr.getvalue()
    device.close()
    retstr.close()
    return text

In [4]:
files = collect_pdfs()

Could not download  http://proceedings.mlr.press/v70/arık17a/arık17a-supp.pdf


In [18]:
# write out all the pdf content into a text file
# avoids having to do this multiple times
    
with open('training_words.txt', 'w') as txt_file:
    for pdf in files:
        text = pdf_to_text(pdf)
        txt_file.write(text)
        txt_file.write('\n\n')

In [69]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

vectorizer = CountVectorizer()
training_words = None

with open('training_words.txt', 'r') as train:
    training_words = train.read()

c = Counter([x.lower() for x in training_words.split()])

In [70]:
counts = {k:v for k, v in sorted(c.items(), key=lambda x: x[1], reverse=True) if k.isalpha() and len(k) > 1}

In [71]:
top10 = dict(sorted(counts.items(), key=lambda x: x[1], reverse = True)[:10])
print(top10)

{'the': 205147, 'of': 101886, 'and': 87289, 'in': 68292, 'to': 64706, 'is': 55298, 'for': 51197, 'we': 50972, 'that': 36466, 'with': 29882}


In [72]:
import numpy as np

total_observed = sum(v for _,v in counts.items())
entropy = 0
for _, obs in counts.items():
    entropy += (obs/total_observed) * np.log2(obs/total_observed)
entropy = -entropy
print(entropy)

9.708366314011496


In [75]:
probabilities = []
for word, obs in counts.items():
    probabilities.extend([word] * obs)
print(len(probabilities))

for i in range(100):
    num = np.random.uniform(0, total_observed)
    print(probabilities[int(num)])

2643672
in
local
zhang
using
jl
two
et
sales
here
the
take
and
is
multiscale
of
for
number
decision
was
are
et
bringing
royal
ranges
to
the
is
of
tried
journal
formal
cab
according
with
guaranteed
from
he
can
as
and
course
literature
axis
with
the
the
is
structured
to
all
exercise
powerful
miny
evaluations
chosen
the
maximization
intervention
set
factorization
our
in
iteration
and
the
we
changing
result
variational
in
and
using
on
is
and
fourth
an
across
the
decision
assumed
using
due
baselines
statement
be
rectangular
spirit
on
in
to
the
of
saga
it
the
introduction
using
clamping
to


## Question 3

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr

%matplotlib inline
train = pd.read_csv('./kaggle/train.csv')
test = pd.read_csv('./kaggle/test.csv')

all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition']))

matplotlib.rcParams['figure.figsize'] = (12.0, 6.0)
prices = pd.DataFrame({"price":train["SalePrice"], "log(price + 1)":np.log1p(train["SalePrice"])})
prices.hist()

In [ ]:
all_data.head()

In [ ]:
train["SalePrice"] = np.log1p(train["SalePrice"])

In [ ]:
numeric_feats = train.dtypes[train.dtypes != "object"].index
skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

all_data[skewed_feats] = np.log1p(all_data[skewed_feats])

all_data.head()

In [ ]:
all_data = pd.get_dummies(all_data)
all_data = all_data.fillna(all_data.mean())
all_data.head()

In [ ]:
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y = train.SalePrice

In [ ]:
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score

model_ridge = Ridge(alpha=0.1)
model_ridge.fit(X_train, y)

results = np.expm1(model_ridge.predict(X_test))
results_df = pd.DataFrame(results)
results_df.columns = ['SalePrice']

results_df.index += 1461
results_df.index.name = 'Id'

results_df.to_csv('./kaggle/results.csv')